<a href="https://colab.research.google.com/github/andreldeod/portfolio/blob/main/code/ArcGIS_REST_Server/Download_vector_layer_from_Arc_REST_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading data from ArcGIS Rest Server
## Instructions pulled from [agsout Github](https://github.com/tannerjt/AGStoShapefile) to download Chilean national archive of wetlands [layer](https://arcgis.mma.gob.cl/server/rest/services/SIMBIO/SIMBIO_HUMEDALES/MapServer/0)

notebook by André

#Create a text file called services.txt
You will use this to direct the code what to download, format of text file is:

 [URL of ArcGIS REST Services Directory] | [Layer ID] | [throttle time in milliseconds]

 The text file I used had the following:


In [ ]:
https://arcgis.mma.gob.cl/server/rest/services/SIMBIO/SIMBIO_HUMEDALES/MapServer/0|Inventario_Nacional_de_Humedales|5000

#Terminal set up and run

Agsout is run on the terminal.

1. If you don't already have it, install GDAL.

2. Install nvm-windows (if not already installed):
Download and install from nvm-windows [releases](https://github.com/coreybutler/nvm-windows/releases): nvm-setup.zip


In [ ]:
#@title Code to run in terminal. Install Node.js and agsout
# Install nvm-windows (if not already installed):

#Install Node.js version 6.x:
nvm install 6

#Use Node.js version 6.x:
nvm use 6


#Verify the installation:
node -v

#Install agsout globally:
npm install -g agsout


Run agsout to download data and convert to shapefile or GeoJson

```
    agsout --help
    agsout -s ./services.txt -o ./backupdir -S
    #-s location of services text file
    #-o directory to backup services
    #-S optional shapefile output (requires gdal)
```
If -S not used then file will be downloaded as Geojson.


In [ ]:
#Run your agsout command with the correct paths:
agsout -s "C:\TerraCarbon\chile_salt_marshes\data\services.txt" -o "C:\TerraCarbon\chile_salt_marshes\data\humedales_ArcGISREST" -S



---



---

#Following code is for post-processing, may not be needed
If the agsout process does not fully work, you will end up with a bunch of json data. The following code was written to combine the jsons and convert into a single shapefile. However, agsout should automatically do this if the whole process is able to be completed.

In [ ]:
#@title Run in WSL after opening Docker to connect notebook locally

docker run --gpus=all -p 127.0.0.1:9000:8080 -v /mnt/c/TerraCarbon/chile_salt_marshes:/workspace us-docker.pkg.dev/colab-images/public/runtime

In [ ]:
#@title Imports
import shapefile
import json
import os
import shutil
import geopandas as gpd
import pandas as pd

import requests

In [ ]:
#@title Paths

# Define the paths
partials_dir = ("/workspace/data/humedales_ArcGISREST/"
                "Inventario_Nacional_de_Humedales/partials")

output_dir = ("/workspace/data/humedales_ArcGISREST/"
              "Inventario_Nacional_de_Humedales")

temp_shapefiles_dir = os.path.join(output_dir, "temp_shapefiles")
merged_shapefile_path = os.path.join(output_dir, "merged_humedales.shp")

# Create a temporary directory for shapefiles
os.makedirs(temp_shapefiles_dir, exist_ok=True)

humedales_dir=output_dir

In [ ]:
#@title Check if malformed files


# List to keep track of problematic files
empty_files = []
malformed_files = []

# Read and validate all JSON files
for filename in sorted(os.listdir(partials_dir)):
    if filename.endswith('.json'):
        file_path = os.path.join(partials_dir, filename)
        with open(file_path, 'r') as f:
            try:
                data = json.load(f)
                if not data:
                    empty_files.append(filename)
            except json.JSONDecodeError:
                malformed_files.append(filename)

print(f"Empty files: {empty_files}")
print(f"Malformed files: {malformed_files}")

# Optionally, remove empty or malformed files
for filename in empty_files + malformed_files:
    os.remove(os.path.join(partials_dir, filename))
    print(f"Removed file: {filename}")


Empty files: []
Malformed files: []


In [ ]:
#@title Convert jsons to shapefiles then combine into one shapefile

# List to track mismatched files
mismatched_files = []

# Function to process a single JSON file and create a shapefile
def process_json_to_shapefile(json_file_path, shapefile_path):
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)

    # Create a shapefile writer object
    w = shapefile.Writer(shapefile_path, shapeType=shapefile.POLYGON)

    # Define the fields
    w.field("OBJECTID", "N")
    w.field("COD_HUMEDA", "C", size=50)
    w.field("NOM_HUMDET", "C", size=255)
    w.field("NOM_HUMMASTER", "C", size=255)
    w.field("ORDEN_1", "N")
    w.field("ORDEN_2", "N")
    w.field("ORDEN_3", "N")
    w.field("ORDEN_4", "N")
    w.field("ORDEN_5", "N")
    w.field("HECTAREAS", "F", decimal=10)
    w.field("HLIMITEURBANO", "F", decimal=10)
    w.field("URL_SIMBIO", "C", size=255)
    w.field("COD_HUMMAS", "C", size=50)

    shapes = []
    records = []
    feature_count = 0

    # Iterate over the features and collect shapes and records
    for feature in data["features"]:
        geometry = feature["geometry"]
        properties = feature["properties"]

        # Add the polygon or multipolygon geometry
        if geometry["type"] == "Polygon":
            shapes.append(geometry["coordinates"])
            records.append([
                properties.get("OBJECTID", None),
                properties.get("COD_HUMEDA", None),
                properties.get("NOM_HUMDET", None),
                properties.get("NOM_HUMMASTER", None),
                properties.get("ORDEN_1", None),
                properties.get("ORDEN_2", None),
                properties.get("ORDEN_3", None),
                properties.get("ORDEN_4", None),
                properties.get("ORDEN_5", None),
                properties.get("HECTAREAS", None),
                properties.get("HLIMITEURBANO", None),
                properties.get("URL_SIMBIO", None),
                properties.get("COD_HUMMAS", None)
            ])
            feature_count += 1
        elif geometry["type"] == "MultiPolygon":
            # Flatten a list of multipolygons into a list of polygons
            flattened_coords = [
                polygon
                for multipolygon in geometry["coordinates"]
                for polygon in multipolygon
            ]

            shapes.append(flattened_coords)
            records.append([
                properties.get("OBJECTID", None),
                properties.get("COD_HUMEDA", None),
                properties.get("NOM_HUMDET", None),
                properties.get("NOM_HUMMASTER", None),
                properties.get("ORDEN_1", None),
                properties.get("ORDEN_2", None),
                properties.get("ORDEN_3", None),
                properties.get("ORDEN_4", None),
                properties.get("ORDEN_5", None),
                properties.get("HECTAREAS", None),
                properties.get("HLIMITEURBANO", None),
                properties.get("URL_SIMBIO", None),
                properties.get("COD_HUMMAS", None)
            ])
            feature_count += 1
        else:
            print(f"Skipping non-polygon feature: {geometry['type']}")
            continue

    # Check if the number of shapes and records match
    if len(shapes) != feature_count:
        print(f"Warning: Number of shapes ({len(shapes)}) does not match number"
        "of records ({feature_count}) in {os.path.basename(json_file_path)}")
        mismatched_files.append(json_file_path)
        return

    # Write the shapes and records to the shapefile
    for shape_coords, record in zip(shapes, records):
        w.poly(shape_coords)
        w.record(*record)

    # Save the shapefile
    w.close()

    # Create the projection file
    prj_path = shapefile_path.replace(".shp", ".prj")
    with open(prj_path, "w") as prj:
        epsg = (
            'GEOGCS["WGS 84",'
            'DATUM["WGS_1984",'
            'SPHEROID["WGS 84",6378137,298.257223563]],'
            'PRIMEM["Greenwich",0],'
            'UNIT["degree",0.0174532925199433]]'
        )
        prj.write(epsg)

    print(f"Shapefile saved to {os.path.basename(shapefile_path)}")

# Process each JSON file and create corresponding shapefiles
json_files = sorted([f for f in os.listdir(partials_dir) if f.endswith('.json')])
shapefile_paths = []
for idx, json_file in enumerate(json_files):
    json_file_path = os.path.join(partials_dir, json_file)
    shapefile_path = os.path.join(temp_shapefiles_dir, f"part_{idx}.shp")
    process_json_to_shapefile(json_file_path, shapefile_path)
    shapefile_paths.append(shapefile_path)

# Merge the shapefiles using GeoPandas
gdfs = [gpd.read_file(shapefile_path) for shapefile_path in shapefile_paths]
merged_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

# Save the merged shapefile
merged_gdf.to_file(merged_shapefile_path, driver='ESRI Shapefile')

print(f"Merged shapefile saved to {os.path.basename(merged_shapefile_path)}")

# Delete the temporary shapefiles directory
shutil.rmtree(temp_shapefiles_dir)
print(f"Deleted temporary shapefiles"
      " directory {os.path.basename(temp_shapefiles_dir)}")

# Log mismatched files
if mismatched_files:
    print(f"The following files had mismatched shapes"
          " and records and were skipped:")
    for file in mismatched_files:
        print(os.path.basename(file))


In [ ]:
#@title Save just one shapefile from one json file

# Define the paths
json_file_path = "/workspace/data/humedales_ArcGISREST/Inventario_Nacional_de_Humedales/partials/935.json"
output_dir = "/workspace/data/humedales_ArcGISREST/Inventario_Nacional_de_Humedales"
shapefile_path = os.path.join(output_dir, "935.shp")

# Function to process a single JSON file and create a shapefile
def process_json_to_shapefile(json_file_path, shapefile_path):
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)

    # Create a shapefile writer object
    w = shapefile.Writer(shapefile_path, shapeType=shapefile.POLYGON)

    # Define the fields
    w.field("OBJECTID", "N")
    w.field("COD_HUMEDA", "C", size=50)
    w.field("NOM_HUMDET", "C", size=255)
    w.field("NOM_HUMMASTER", "C", size=255)
    w.field("ORDEN_1", "N")
    w.field("ORDEN_2", "N")
    w.field("ORDEN_3", "N")
    w.field("ORDEN_4", "N")
    w.field("ORDEN_5", "N")
    w.field("HECTAREAS", "F", decimal=10)
    w.field("HLIMITEURBANO", "F", decimal=10)
    w.field("URL_SIMBIO", "C", size=255)
    w.field("COD_HUMMAS", "C", size=50)

    shapes = []
    records = []
    feature_count = 0

    # Iterate over the features and collect shapes and records
    for feature in data["features"]:
        geometry = feature["geometry"]
        properties = feature["properties"]

        # Add the polygon or multipolygon geometry
        if geometry["type"] == "Polygon":
            shapes.append(geometry["coordinates"])
            records.append([
                properties.get("OBJECTID", None),
                properties.get("COD_HUMEDA", None),
                properties.get("NOM_HUMDET", None),
                properties.get("NOM_HUMMASTER", None),
                properties.get("ORDEN_1", None),
                properties.get("ORDEN_2", None),
                properties.get("ORDEN_3", None),
                properties.get("ORDEN_4", None),
                properties.get("ORDEN_5", None),
                properties.get("HECTAREAS", None),
                properties.get("HLIMITEURBANO", None),
                properties.get("URL_SIMBIO", None),
                properties.get("COD_HUMMAS", None)
            ])
            feature_count += 1
        elif geometry["type"] == "MultiPolygon":
            # Flatten a list of multipolygons into a list of polygons
            flattened_coords = [
                polygon
                for multipolygon in geometry["coordinates"]
                for polygon in multipolygon
            ]
            shapes.append(flattened_coords)
            records.append([
                properties.get("OBJECTID", None),
                properties.get("COD_HUMEDA", None),
                properties.get("NOM_HUMDET", None),
                properties.get("NOM_HUMMASTER", None),
                properties.get("ORDEN_1", None),
                properties.get("ORDEN_2", None),
                properties.get("ORDEN_3", None),
                properties.get("ORDEN_4", None),
                properties.get("ORDEN_5", None),
                properties.get("HECTAREAS", None),
                properties.get("HLIMITEURBANO", None),
                properties.get("URL_SIMBIO", None),
                properties.get("COD_HUMMAS", None)
            ])
            feature_count += 1
        else:
            print(f"Skipping non-polygon feature: {geometry['type']}")
            continue

    # Check if the number of shapes and records match
    if len(shapes) != feature_count:
        print(f"Warning: Number of shapes ({len(shapes)}) does not match number"
        "of records ({feature_count}) in {os.path.basename(json_file_path)}")
        return

    # Write the shapes and records to the shapefile
    for shape_coords, record in zip(shapes, records):
        w.poly(shape_coords)
        w.record(*record)

    # Save the shapefile
    w.close()

    # Create the projection file
    prj_path = shapefile_path.replace(".shp", ".prj")
    with open(prj_path, "w") as prj:
        epsg = (
            'GEOGCS["WGS 84",'
            'DATUM["WGS_1984",'
            'SPHEROID["WGS 84",6378137,298.257223563]],'
            'PRIMEM["Greenwich",0],'
            'UNIT["degree",0.0174532925199433]]'
        )
        prj.write(epsg)

    print(f"Shapefile saved to {os.path.basename(shapefile_path)}")

# Process the specific JSON file
process_json_to_shapefile(json_file_path, shapefile_path)


Shapefile saved to 935.shp




---
#Additional potentially helpful code


In [ ]:
#@title Extract coded values for ORDEN fields from ArcGIS REST service
#Script to extract coded values for ORDEN fields from ArcGIS REST service
#import requests

# URL of the ArcGIS REST service layer
url = ('https://arcgis.mma.gob.cl/server/rest/services/SIMBIO/'
       'SIMBIO_HUMEDALES/MapServer/0')

# Function to get layer information
def get_layer_info(url):
    try:
        response = requests.get(f'{url}?f=json')
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching layer info: {e}")
        return None

# Function to print coded values for a field
def print_coded_values(field_name, layer_info):
    if not layer_info:
        return
    for field in layer_info['fields']:
        if field['name'] == field_name:
            print(f"Coded values for {field_name}:")
            if 'domain' in field and 'codedValues' in field['domain']:
                for coded_value in field['domain']['codedValues']:
                    code = coded_value['code']
                    description = coded_value['name']
                    print(f"  {code}: {description}")
            else:
                print("  No coded values found for this field.")

# Fetch layer info
layer_info = get_layer_info(url)

# Print coded values for each ORDEN field
print_coded_values('ORDEN_1', layer_info)
print_coded_values('ORDEN_2', layer_info)
print_coded_values('ORDEN_3', layer_info)
print_coded_values('ORDEN_4', layer_info)
print_coded_values('ORDEN_5', layer_info)

Coded values for ORDEN_1:
  10: ARTIFICIALES
  20: CONTINENTALES
  30: MARINOS Y COSTEROS
  999: SIN CLASIFICAR
Coded values for ORDEN_2:
  10: ALMACENAMIENTO
  20: LACUSTRES
  30: PALUSTRES
  40: RIBERENOS
  50: ESTUARINOS
  60: MARINOS
  70: LAGO SALADO
  80: EXPLOTACION DE SAL
  90: LAGUNAR
  999: SIN CLASIFICAR
Coded values for ORDEN_3:
  10: ALMACENAMIENTO
  20: BOSCOSOS
  30: EMERGENTES
  40: EXPLOTACION DE SAL
  50: INTERMAREALES
  60: LAGO SALADO
  70: LAGUNAR
  80: PERMANENTES
  90: SUBMAREALES
  100: TEMPORALES
  999: SIN CLASIFICAR
Coded values for ORDEN_4:
  10: ANDINOS
  20: DELTAS INTERIORES
  30: EMBALSE
  40: ESTACIONALES
  50: ESTERO
  60: ESTUARIOS
  70: INTERMAREALES
  80: IRREGULARES
  90: LAGO
  100: LAGO SALADO
  110: LAGUNA
  120: LAGUNA SALADA
  130: MALLINES
  140: PERMANENTES
  150: PLAYAS
  160: RIO
  170: SALARES
  180: SALINAS
  190: TRANQUE
  200: TURBERAS
  999: SIN CLASIFICAR
Coded values for ORDEN_5:
  10: BOFEDAL
  20: DELTAS INTERIORES
  30: EMBALSE
 

In [ ]:
#@title Replace ORDEN attribute code with names
# This script replaces number attributes in ORDEN_ fields with their coded
# values (with only the first letter capitalized and including translations)
# in a shapefile.

#import geopandas as gpd

humedales_dir = ("/workspace/data/humedales_ArcGISREST/"
              "Inventario_Nacional_de_Humedales")

# Path to the input shapefile
input_shapefile = (f'{humedales_dir}/Inventario_Nacional_de_Humedales_2024'
                   '/Inventario_Nacional_de_Humedales.shp')

# Path to the output shapefile
output_shapefile =(f'{humedales_dir}/Inventario_Nacional_de_Humedales_withNames'
                    '/Inventario_Nacional_de_Humedales.shp')

# Dictionary with coded values and their translations for each ORDEN field
orden_codes = {
    'ORDEN_1': {
        10: 'Artificiales - Artificial', 20: 'Continentales - Continental',
        30: 'Marinos y costeros - Marine and coastal',
        999: 'Sin clasificar - Unclassified'
    },
    'ORDEN_2': {
        10: 'Almacenamiento - Storage', 20: 'Lacustres - Lacustrine',
        30: 'Palustres - Palustrine', 40: 'Riberenos - Riparian',
        50: 'Estuarinos - Estuarine', 60: 'Marinos - Marine',
        70: 'Lago salado - Salt lake',
        80: 'Explotacion de sal - Salt exploitation',
        90: 'Lagunar - Lagoonal', 999: 'Sin clasificar - Unclassified'
    },
    'ORDEN_3': {
        10: 'Almacenamiento - Storage', 20: 'Boscosos - Forested',
        30: 'Emergentes - Emergent',
        40: 'Explotacion de sal - Salt exploitation',
        50: 'Intermareales - Intertidal', 60: 'Lago salado - Salt lake',
        70: 'Lagunar - Lagoonal', 80: 'Permanentes - Permanent',
        90: 'Submareales - Subtidal', 100: 'Temporales - Temporary',
        999: 'Sin clasificar - Unclassified'
    },
    'ORDEN_4': {
        10: 'Andinos - Andean', 20: 'Deltas interiores - Interior deltas',
        30: 'Embalse - Reservoir', 40: 'Estacionales - Seasonal',
        50: 'Estero - Estuary', 60: 'Estuarios - Estuaries',
        70: 'Intermareales - Intertidal', 80: 'Irregulares - Irregular',
        90: 'Lago - Lake', 100: 'Lago salado - Salt lake',
        110: 'Laguna - Lagoon', 120: 'Laguna salada - Salt lagoon',
        130: 'Mallines - Mallines', 140: 'Permanentes - Permanent',
        150: 'Playas - Beaches', 160: 'Rio - River',
        170: 'Salares - Salt flats',
        180: 'Salinas - Salt mines', 190: 'Tranque - Dam',
        200: 'Turberas - Peatlands',
        999: 'Sin clasificar - Unclassified'
    },
    'ORDEN_5': {
        10: 'Bofedal - Bofedal', 20: 'Deltas interiores - Interior deltas',
        30: 'Embalse - Reservoir', 40: 'Estacionales - Seasonal',
        50: 'Estero - Estuary', 60: 'Estuarios - Estuaries',
        70: 'Geiser - Geyser', 80: 'Intermareales - Intertidal',
        90: 'Irregulares - Irregular', 100: 'Lago - Lake',
        110: 'Lago salado - Salt lake', 120: 'Laguna - Lagoon',
        130: 'Laguna salada - Salt lagoon', 140: 'Mallines - Mallines',
        150: 'Perdida - Lost', 160: 'Permanentes - Permanent',
        170: 'Playas - Beaches', 180: 'Rio - River',
        190: 'Salares - Salt flats',
        200: 'Salinas - Salt mines', 210: 'Tranque - Dam',
        220: 'Turberas - Peatlands', 230: 'Vega - Meadow',
        240: 'Turbera de sphagnum - Sphagnum peat bog',
        250: 'Turbera natural - Natural peat bog',
        260: 'Turbera en altura - Highland peat bog',
        270: 'Turbera antropogenica - Anthropogenic peat bog',
        280: 'Turbera en interfase estuarina - Estuarine interface peat bog',
        290: 'Turbera explotada - Exploited peat bog',
        999: 'Sin clasificar - Unclassified'
    }
}

# Function to replace the numeric codes with their corresponding values
def replace_orden_codes(df, field, codes):
    df[field] = df[field].map(codes).fillna(df[field])
    return df

# Load the shapefile
gdf = gpd.read_file(input_shapefile)

# Replace the codes for each ORDEN field
for field in orden_codes.keys():
    if field in gdf.columns:
        gdf = replace_orden_codes(gdf, field, orden_codes[field])

# Save the modified shapefile
gdf.to_file(output_shapefile)

print("Shapefile processing completed successfully.")


Shapefile processing completed successfully.


In [ ]:
#@title Get bounding box of geojson

# Function to get the bounding box from the AOI GeoJSON file
def get_aoi_extent(aoi_path):
    gdf = gpd.read_file(aoi_path)
    bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]
    return bounds

# URL for the ArcGIS REST service query
url = "https://arcgis.mma.gob.cl/server/rest/services/SIMBIO/SIMBIO_HUMEDALES/MapServer/0/query"

# Path to the AOI GeoJSON file
aoi_path = "/workspace/data/AOI/AOI.geojson"

# Get the bounding box from the AOI
minx, miny, maxx, maxy = get_aoi_extent(aoi_path)

# Parameters for the initial query to get the object IDs
initial_params = {
    "geometry": f"{minx},{miny},{maxx},{maxy}",
    "geometryType": "esriGeometryEnvelope",
    "spatialRel": "esriSpatialRelIntersects",
    "where": "1=1",
    "returnIdsOnly": "true",
    "f": "json"
}

# Send the initial request to get object IDs
initial_response = requests.get(url, params=initial_params)

# Debug: Print the initial response content
print("Initial Response Status Code:", initial_response.status_code)
print("Initial Response Text:", initial_response.text)

if initial_response.status_code == 200:
    object_ids = initial_response.json().get('objectIds', None)

    if object_ids is None:
        print("No objectIds returned in the response.")
    else:
        max_record_count = 1000  # Set the maxRecordCount parameter

        all_features = []

        def fetch_data(subset_ids, retries=3):
            params = {
                "objectIds": ",".join(map(str, subset_ids)),
                "outFields": "*",
                "f": "geojson",
                "returnGeometry": "true"
            }

            for attempt in range(retries):
                response = requests.get(url, params=params)
                if response.status_code == 200:
                    return response.json().get('features', [])
                else:
                    print(f"Failed attempt {attempt + 1} to download data chunk. HTTP status code: {response.status_code}")
                    time.sleep(2)  # Wait before retrying

            return []

        # Paginate through the records
        for i in range(0, len(object_ids), max_record_count):
            subset_ids = object_ids[i:i+max_record_count]
            features = fetch_data(subset_ids)
            all_features.extend(features)

        if all_features:
            geojson_data = {
                "type": "FeatureCollection",
                "features": all_features
            }
            # Save the combined features as a GeoJSON file in /workspace directory
            output_path = "/workspace/Inventario_Nacional_de_Humedales_clipped.geojson"
            with open(output_path, "w") as file:
                json.dump(geojson_data, file)
            print(f"Data downloaded successfully and saved to {output_path}")
        else:
            print("No data was downloaded.")
else:
    print(f"Failed to retrieve object IDs. " \
          f"HTTP status code: {initial_response.status_code}")
    print(f"Response: {initial_response.text}")


Initial Response Status Code: 200
Initial Response Text: {"objectIdFieldName":"OBJECTID","objectIds":null}
No objectIds returned in the response.


In [ ]:
#@title Print bounding box of geojson

# Path to the AOI GeoJSON file
aoi_path = "/workspace/data/AOI/AOI.geojson"

# Load AOI GeoJSON file and get bounding box
gdf = gpd.read_file(aoi_path)
bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]

print("AOI Bounding Box:", bounds)


AOI Bounding Box: [-8330898.5987 -5475824.89   -7884401.6909 -4361699.6819]
